<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/CryptoPulse3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing necessary libraries
!pip install yfinance pandas numpy scikit-learn matplotlib ta

import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from ta.trend import MACD, SMAIndicator
from ta.momentum import StochasticOscillator, RSIIndicator
import matplotlib.pyplot as plt

# Define the stock symbols for American Express, Boeing, McDonald's, Exxon Mobil, FedEx, Netflix, Intel, BTC and others
stock_symbols = [
    'AXP', 'BA', 'MCD', 'XOM', 'FDX', 'NFLX', 'INTC',
    'JNJ', 'TSLA', 'V', 'PFE', 'BABA', 'CSCO', 'MSFT',
    'C', 'AMZN', 'TWTR', 'BTC-USD'  # Added BTC
]
time_interval = '1h'  # 1-hour intervals

# Initialize an empty list to store the results
results = []

# Function to process each stock and predict buy/sell for the next hour
def predict_for_stock(stock_symbol):
    # Fetch fresh data from Yahoo Finance for the last 3 months (to speed up)
    data = yf.download(stock_symbol, period='3mo', interval=time_interval)

    # Feature Engineering - Moving Averages, MACD, Stochastic Oscillator, RSI

    # 1. Moving Averages (SMA - Simple Moving Averages)
    data['SMA_10'] = SMAIndicator(data['Close'], window=10).sma_indicator()
    data['SMA_30'] = SMAIndicator(data['Close'], window=30).sma_indicator()

    # 2. MACD (Moving Average Convergence Divergence)
    macd = MACD(close=data['Close'], window_slow=26, window_fast=12, window_sign=9)
    data['MACD'] = macd.macd()
    data['MACD_signal'] = macd.macd_signal()

    # 3. Stochastic Oscillator
    stoch = StochasticOscillator(high=data['High'], low=data['Low'], close=data['Close'], window=14, smooth_window=3)
    data['stoch_k'] = stoch.stoch()
    data['stoch_d'] = stoch.stoch_signal()

    # 4. Relative Strength Index (RSI)
    rsi = RSIIndicator(close=data['Close'], window=14)
    data['RSI'] = rsi.rsi()

    # Drop NaN values (as some indicators are based on rolling windows)
    data.dropna(inplace=True)

    # Define buy/sell signal: Buy if MACD > Signal and SMA_10 > SMA_30, otherwise sell
    def buy_sell_signal(row):
        if row['MACD'] > row['MACD_signal'] and row['SMA_10'] > row['SMA_30']:
            return 1  # Buy
        else:
            return 0  # Sell

    data['Signal'] = data.apply(buy_sell_signal, axis=1)

    # Check if there are enough samples after feature engineering
    if data.shape[0] > 10:  # Make sure there are enough rows

        # Define features (X) and target variable (y)
        X = data[['SMA_10', 'SMA_30', 'MACD', 'MACD_signal', 'stoch_k', 'stoch_d', 'RSI']]
        y = data['Signal']

        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

        # Train a simpler RandomForestClassifier without GridSearchCV
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # Predict the next hour signal using the last available data point
        next_hour_data = data.iloc[-1][['SMA_10', 'SMA_30', 'MACD', 'MACD_signal', 'stoch_k', 'stoch_d', 'RSI']].values.reshape(1, -1)
        next_hour_prediction = model.predict(next_hour_data)[0]

        # Output decision based on next hour prediction
        if next_hour_prediction == 1:
            decision = "BUY"
        else:
            decision = "SELL"

        # Append the result to the list
        results.append({
            'Stock': stock_symbol,
            'Recommendation': decision
        })
    else:
        # Append no data available if the dataset is insufficient
        results.append({
            'Stock': stock_symbol,
            'Recommendation': 'NOT ENOUGH DATA'
        })

# Loop through each stock and make the prediction
for stock_symbol in stock_symbols:
    predict_for_stock(stock_symbol)

# Convert the results to a DataFrame for better display
results_df = pd.DataFrame(results)

# Display the results in a nice table
print("Next Hour Predictions:")
print(results_df)

# Optionally display as a nicely formatted table (if you're using Jupyter Notebook)
try:
    import ace_tools as tools  # For Jupyter integration, if available
    tools.display_dataframe_to_user(name="Stock and BTC Buy/Sell Predictions", dataframe=results_df)
except ImportError:
    pass  # This is optional for specific environments only


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=255d5df2e02e19598878ede1423744bb4c646adfba99739288ef7079dacc3773
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
 

Next Hour Predictions:
      Stock   Recommendation
0       AXP             SELL
1        BA             SELL
2       MCD             SELL
3       XOM              BUY
4       FDX             SELL
5      NFLX              BUY
6      INTC              BUY
7       JNJ             SELL
8      TSLA             SELL
9         V             SELL
10      PFE             SELL
11     BABA              BUY
12     CSCO             SELL
13     MSFT             SELL
14        C             SELL
15     AMZN             SELL
16     TWTR  NOT ENOUGH DATA
17  BTC-USD             SELL


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
